# Import packages

In [ ]:
import os
import re
import spacy
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

# Load data

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Load the environment variables
load_dotenv()

# Get the paths to the data
data_dir = os.getenv("DATA_DIR")
articles_dir = os.path.join(data_dir, "articles")

# Load the main dataset
df = pd.read_csv(f"{data_dir}/article_info_V2.csv", index_col=0, parse_dates=[1])

# Load the external dataset
external_df = pd.read_csv(f"{data_dir}/other-articles.csv", index_col=0, parse_dates=[1], sep=";")

# Cleaning

In [ ]:
# Drop unused columns
df.drop(columns=["Author", "Type", "Keywords"], inplace=True)

# Drop rows with empty date values
df.dropna(subset=["Date"], inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

# Remove the url column from the external dataset
external_df.drop(columns=["Url"], inplace=True)

# Merging datasets

In [ ]:
# Add the external dataset to the main dataset
df = pd.concat([df, external_df])

# Reset the index
df.reset_index(drop=True, inplace=True)

# Clear the external dataset variable
del external_df

# Remove duplicates

In [ ]:
# Count the number of entries before duplicate removal
entries_count = len(df)

# remove duplicate entries by title and date
df.drop_duplicates(subset=["Title", "Date"], keep="first", inplace=True)

# Drop the date column
df.drop(columns=["Date"], inplace=True)

# Print the number of deleted entries
print(f"Removed {entries_count - len(df)} duplicate entries.")

# Clear the entries count variable
del entries_count

# Parse target

In [ ]:
# Define functions to parse target
def clean_string(str):
    # Make the string lowercase
    str = str.lower()
    # Remove all non-alphanumeric characters
    str = re.sub(r"[^\w\s]", "", str)
    # Remove the leading and trailing spaces
    return str.strip()

def parse_tags(str):
    # Split the string on each comma
    raw_list = str.split(",")
    # Clean every string in the list
    return list(map(clean_string, raw_list))

In [ ]:
# Convert the raw string values of the Tags column to lists of strings
df["Tags"] = df["Tags"].apply(parse_tags)

In [ ]:
# Clear the cleaning and parsing functions
del clean_string, parse_tags

# Load articles

In [ ]:
# Article loading function
def load_article(title):
    # Create filename from title
    file_name = f"{title}.txt"
    # Get the path of the article
    file_path = os.path.join(articles_dir, file_name)
    try:
        # Attempt to read the article
        file = open(file_path, "r", encoding="utf-8")
    except:
        # If the article could not be read, return None
        return None
    # Read the contents of the article
    contents = file.read()
    # Close the file
    file.close()
    # Return the contents of the article
    return contents

In [ ]:
# Load the articles
df["Contents"] = df["Title"].apply(load_article)

# Clear the article loading function
del load_article

In [ ]:
# Count the number of entries in the dataframe
entries_count = len(df)

# Remove rows without article contents
df.dropna(subset=["Contents"], inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

# Print the number of deleted entries
print(f"Removed {entries_count - len(df)} entries without article contents.")

# Clear the entries count variable
del entries_count

# Create categories from tags

In [ ]:
unique_tags = []
for tag_list in df["Tags"]:
    for tag in tag_list:
        if tag not in unique_tags:
            unique_tags.append(tag)

In [ ]:
# Load the custom categories dataset
custom_categories = pd.read_csv(f"{data_dir}/new-categories.csv")

# After receiving feedback from SKC, we decided to drop the following categories:
custom_categories.drop(columns=["People", "Places", "Criminal groups"], inplace=True)

# Define a function to parse the custom categories into a list of strings
def load_category(name):
    try:
        return custom_categories[name].dropna().tolist()
    except:
        return None

In [ ]:
# Initiate the category dictionary
categories = {}

# Iterate over these custom_categories columns
for category_name in custom_categories.columns:
    # Load the category
    categories[category_name] = load_category(category_name)

In [ ]:
# Create a new column for the category
df["Category"] = ""

# Iterate over the dataframe
for index, row in df.iterrows():
    # Get the tags of the current row
    tag_list = row["Tags"]
    # Create a dictionary for the counters of the categories for this article
    counters = {}
    # Iterate over every tag in the tag list
    for tag in tag_list:
        # Iterate over the categories
        for category_name in categories:
            # Get the current category
            category = categories[category_name]
            # Check if the tag is in the category list
            if tag in category:
                # If it is, add it to the counters dictionary
                if category_name not in counters:
                    counters[category_name] = 1
                else:
                    counters[category_name] += 1    
    # Get the category with the highest counter and assign it to the row
    if len(counters) == 0:
        row["Category"] = "Other"
    else:
        row["Category"] = max(counters, key=counters.get)

# Drop the tags column

In [ ]:
df.drop(columns=["Tags"], inplace=True)

In [ ]:
# drop rows with the other category
df.drop(df[df["Category"] == "Other"].index, inplace=True)

# Visualizations

In [ ]:
category_counts = df["Category"].value_counts()
plt.figure(figsize=(12, 8))
plt.title("Category distribution")
plt.ylabel("Number of articles")
plt.bar(category_counts.index, category_counts.values)
plt.show()

In [ ]:
df.head(10)

In [ ]:
import nltk
import unicodedata
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download("punkt")
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def lemmatize_sentences(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        words = [word for word in words if not word.isdigit()]
        words = [word for word in words if word not in stop_words]
        words = [lemmatizer.lemmatize(word) for word in words]
        # Remove words with less than 4 characters
        words = [word for word in words if len(word) > 3]
        if len(words) > 0:
            cleaned_sentences.append(words)
    return cleaned_sentences

def text_to_sentences(text):
    # Make text lowercase
    text = text.lower()
    # Replace non ascii characters with their ascii counterpart
    text = unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8", "ignore")
    # Remove digits from the text
    text = re.sub(r"\d+", "", text)
    # Remove words with non-alphanumeric characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # Remove double whitespace
    text = re.sub(r"\s+", " ", text)
    # Split the text into sentences
    sentences = nltk.sent_tokenize(text)
    # Lemmatize the sentences
    sentences = lemmatize_sentences(sentences)
    # Remove empty sentences
    sentences = list(filter(None, sentences))
    # Join the words in sentences together
    sentences = list(map(lambda x: " ".join(x), sentences))
    # Return the list of sentences
    return sentences

df["Sentences"] = df["Contents"].apply(text_to_sentences)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

documents = df["Sentences"].apply(lambda x: " ".join(x)).tolist()

tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(documents)
feature_names = tfidf_vectorizer.get_feature_names_out()

In [ ]:
vector_df = pd.DataFrame(values.toarray(), columns=feature_names)

# Check if column has more than 100 values above 0
def check_column(column):
    count = column.value_counts()[column.value_counts() > 0].count()
    return count >= 20

columns_to_drop = list(filter(lambda x: not check_column(vector_df[x]), vector_df.columns))

# apply check column to all columns and drop it if it doesn't pass the check
vector_df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

X = vector_df
y = pd.get_dummies(df["Category"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [ ]:
def specificity_score(target, predictions):
    return 1 - recall_score(target, predictions, average="weighted")

def get_metrics(target, predictions):
    return {
        "recall": recall_score(target, predictions, average="weighted"),
        "precision": precision_score(target, predictions, average="weighted"),
        "accuracy": accuracy_score(target, predictions),
        "specificity": specificity_score(target, predictions),
        "f1": f1_score(target, predictions, average="weighted"),
    }

def print_metrics(target, predictions):
    metrics = get_metrics(target=target, predictions=predictions)
    for metric in metrics:
        str_metric = "{:.2f}".format(metrics[metric] * 100)+"%"
        print(f"{metric}: {str_metric}")

In [ ]:
# import decision tree classifier
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc = dtc.fit(X_train, y_train)
y_pred_dtc = dtc.predict(X_test)

# Print the metrics of the model's performance
print_metrics(y_test, y_pred_dtc)

In [ ]:
# from spacy.matcher import Matcher
# matcher = Matcher(nlp.vocab)

# # Create a new column called Keywords
# df["Keywords"] = ""

# # Write multiple patterns to find if an article contains cocaine etc.
# pattern = [
# # Drugs
# [{"LEMMA": "drug"},{"POS": "NOUN", "OP": "?"}],
# [{"POS": "ADJ", "OP": "?"},{"TEXT": "cocaine"},{"POS": "NOUN", "OP": "?"}],
# # Environmental crimes
# [{"LEMMA": "deforestation"}],
# [{"LEMMA": "environment"}],
# [{"LEMMA": "illegal"},{"POS": "NOUN", "OP": "?"}],
# # Violence
# [{"LEMMA": "violence"}],
# [{"TEXT": "violent"},{"POS": "NOUN", "OP": "?"}],
# [{"LEMMA": "kidnap"}],
# [{"POS": "NOUN", "OP": "?"},{"LEMMA": "threat"}],
# [{"LEMMA": "murder"}],
# [{"LEMMA": "shoot"}],
# [{"TEXT": "death"}],
# [{"TEXT": "assassin"}],
# [{"LEMMA": "robbery"}],
# [{"TEXT": "hitman"}],
# [{"TEXT": "dead"}],
# [{"TEXT": "execution"}],
# [{"POS": "NOUN", "OP": "?"},{"LEMMA": "killing"}],
# # Weapons
# [{"POS": "ADJ", "OP": "?"},{"LEMMA": "weapon"},{"POS": "NOUN", "OP": "?"}],
# [{"POS": "VERB", "OP": "?"},{"LEMMA": "weapon"}],
# [{"LEMMA": "arms"},{"POS": "NOUN", "OP": "?"}],
# [{"LEMMA": "gun"},{"POS": "NOUN", "OP": "?"}],
# [{"TEXT": "AK-47"}],
# [{"LEMMA": "explosive"}],
# [{"POS": "NOUN", "OP": "?"},{"LEMMA": "rifle"}],
# # Trafficking
# [{"POS": "ADJ", "OP": "?"},{"LEMMA": "human"},{"POS": "NOUN", "OP": "?"}],
# [{"POS": "NOUN", "OP":"?"},{"LEMMA": "trafficking"}],
# [{"TEXT": "traffickers"}],
# # Money
# [{"POS": "ADJ", "OP": "?"},{"LEMMA": "laundering"},{"POS": "NOUN", "OP": "?"}],
# [{"TEXT": "fraudulent"}],
# [{"LEMMA": "fraude"}],
# [{"LEMMA": "money"}],
# [{"LEMMA": "corruption"}],
# # Other
# ]

# # Add the pattern to the matcher and 
# matcher.add("DETERMINE CATEGORY", [pattern][0])

# # Set the index of the article content
# i = 0
# # Loop over all articles contents
# for content in df["Contents"][0:10]:
#     doc = nlp(content)
#     # Apply the matcher to the doc
#     matches = matcher(doc)
#     # Create an empty array for all keywords per article
#     all_generated_keywords = []
#     # Iterate over the matches and append them to an array
#     for match_id, start, end in matches:
#         all_generated_keywords.append(doc[start:end].text)
#     # Remove duplicates from array and assign them to variable
#     duplicates_removed =  list(dict.fromkeys(all_generated_keywords))
#     # Append keywords without duplicates to the Keywords column in the dataframe
#     df["Keywords"][i] = duplicates_removed
#     # Increase the index of the article content
#     i += 1